In [1]:
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.3.0


# Load Data

In [2]:
train_df = pd.read_csv('../data/AI_ML_Challenge_Training_Data_Set_1_v1.csv')
val_df = pd.read_csv('../data/AI_ML_Challenge_Validation_Data_Set_v1.csv')

In [3]:
val_df

,Clause ID,Clause Text
0,94,\tthe Customer does not make any admissions (s...
1,7028,Requests. Company will notify Customer before ...
2,9048,We sometimes release beta versions of our webs...
3,7755,Termination without Cause. Customer may termin...
4,1145,1.8 “Term” means the term of this Agreement as...
...,...,...
1386,6590,OWNERSHIP; U.S. GOVERNMENT RESTRICTED RIGHTS .
1387,2333,5. User Conduct.\n\n5.1 Responsible Use. The C...
1388,2489,6.1 During the Warranty Period COMPANY shall p...
1389,2548,7. Assignment of Duties. Vendor may assign its...


In [4]:
train_df['Clause Text'][0]

'18. Governing Law: This Agreement shall be governed by and interpreted in accordance with the Federal laws of theUnited States, without reference to conflict-of-laws principles. If for any reason a court of competent jurisdiction finds any provision of this Agreement to be unenforceable, that provision will be enforced to the maximum extent possible to effectuate the intent of the parties, and the remainder of this Agreement will continue in full force and effect. This Agreement shall not be governed by the United Nations Convention on Contracts for the International Sale of Goods. Buyer agrees that exclusive jurisdiction for any dispute arising out of or relating to this Agreement lies within the venue mandated by applicable Federal law.'

In [5]:
train_df

,Clause ID,Clause Text,Classification
0,1588,18. Governing Law: This Agreement shall be gov...,0
1,1146,"1.8 Modification. We may modify, update, or di...",1
2,4792,Except as otherwise expressly provided in this...,0
3,2759,8.3. The benefit and burdens of this Ag...,1
4,4400,DEFINITIONS,0
...,...,...,...
7874,7905,The END-USER is hereby granted by COMPANY a li...,0
7875,8456,"This End User License Agreement (""EULA"") is a ...",1
7876,295,- The financial cumulative liability of COMPAN...,0
7877,92,\tThe Customer acknowledges that all Intellect...,0


In [6]:
import numpy as np

# Data Profiling with Pandas Profiling

In [15]:
from pandas_profiling import ProfileReport
#ProfileReport(train_df)

# Clause Length
Bert max_length for classification 512 words

In [12]:
train_df['clause_length'] = train_df['Clause Text'].str.split().apply(len)

In [13]:
train_df.clause_length.describe()

count    7879.000000
mean       79.855185
std       131.183433
min         0.000000
25%        19.000000
50%        49.000000
75%       101.000000
max      7347.000000
Name: clause_length, dtype: float64

In [14]:
from scipy import stats

for words in [64, 128, 256, 512, 1024]:
    print(f'{words}: {stats.percentileofscore(train_df.clause_length, words)}')

64: 58.81457037695139
128: 82.07259804543723
256: 94.73283411600457
512: 99.23848204086813
1024: 99.94923213605787


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, add_special_tokens=True,
                                                max_length=512, pad_to_max_length=True)

In [ ]:
targets = train_df.pop('Classification')
targets[:2]

0    0
1    1
Name: Classification, dtype: int64

In [ ]:
sentences = train_df.pop('Clause Text')
sentences[:2]

0    18. Governing Law: This Agreement shall be gov...
1    1.8 Modification. We may modify, update, or di...
Name: Clause Text, dtype: object

In [ ]:
for p in test_dataset.take(2):
  print(p[0])

{'input_ids': <tf.Tensor: shape=(512,), dtype=int32, numpy=
array([  101,  1020,  1012,  5860, 19771,  5017,  1997, 10943,  3111,
        1012,  2017, 13399,  1998,  5993,  2008,  1996,  4132,  1010,
        2164,  2035,  4180,  4838,  2045,  2378,  2030, 11570,  8558,
        1010,  2003,  3024,  2006,  2019,  1523,  2004,  2003,  1521,
        1998,  1523,  2004,  2800,  1524,  3978,  1010,  1998,  2008,
        2115,  2224,  1997,  2030, 17975,  2588,  1996,  4132,  1998,
        2151,  2353,  2283,  4180, 11570,  8558,  2003,  2012,  2115,
        7082,  3891,  1998, 19258,  1012,  2012,  1004,  1056,  1998,
        2049, 21315,  1010, 20141,  1998,  5622, 19023,  5668,  2182,
        3762,  5860, 19771,  2213,  2151,  1998,  2035, 15066,  1010,
       10943,  3111,  1998, 19739, 20486,  7368,  4953,  1996,  4132,
        1998,  2353,  2283,  4180,  1010,  3251,  8700,  1010,  4671,
        1010, 13339,  2030, 15201,  1010,  1998,  3251, 17707,  2011,
        2375,  1010, 11671,  1

In [ ]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
      return {
          "input_ids": input_ids,
          "token_type_ids": token_type_ids,
          "attention_mask": attention_masks,
      }, label
def encode_examples(sentences, targets, limit=-1, max_length=512):
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    #if (limit > 0):
    #ds = ds.take(limit)

    for review, label in zip(sentences.values, targets.values):
        if not review:
            continue
        bert_input = tokenizer.encode_plus(review, add_special_tokens=True, max_length=max_length, 
                                           padding='max_length', truncation=True)

        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [ ]:
data = encode_examples(sentences, targets)

In [ ]:
DATASET_SIZE = len(train_df)
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = data
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [ ]:
train_data = train_dataset.shuffle(100).batch(8)


In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
from tensorflow_probability import stats as tfs

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5
# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1

# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# optimizer Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalCrossentropy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_75', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_history = model.fit(train_data, epochs=2, validation_data=val_dataset.batch(32))

Epoch 1/2
690/690 [==============================] - 734s 1s/step - loss: 0.3964 - accuracy: 2.2319 - val_loss: 0.3241 - val_accuracy: 1.8943
Epoch 2/2
690/690 [==============================] - 742s 1s/step - loss: 0.2852 - accuracy: 1.6087 - val_loss: 0.2946 - val_accuracy: 1.8728


In [ ]:
!mkdir /content/ai-ml-challenge-2020/models/

mkdir: cannot create directory ‘/content/ai-ml-challenge-2020/models/’: File exists


In [ ]:
model.save_pretrained('/content/ai-ml-challenge-2020/models/')

In [ ]:
bert_history.history

{'accuracy': [2.2318522930145264, 1.608742117881775],
 'loss': [0.39644166827201843, 0.28521427512168884],
 'val_accuracy': [1.8942794799804688, 1.8727525472640991],
 'val_loss': [0.3240688145160675, 0.2945655584335327]}

In [ ]:
from transformers import pipeline

pipeline('sentiment-analysis', model=model, tokenizer=tokenizer,)(sentences.values[7])

[{'label': 'LABEL_0', 'score': 0.9995656609535217}]

In [ ]:
sentences

0       18. Governing Law: This Agreement shall be gov...
1       1.8 Modification. We may modify, update, or di...
2       Except as otherwise expressly provided in this...
3       8.3.        The benefit and burdens of this Ag...
4                                             DEFINITIONS
                              ...                        
7874    The END-USER is hereby granted by COMPANY a li...
7875    This End User License Agreement ("EULA") is a ...
7876    - The financial cumulative liability of COMPAN...
7877    \tThe Customer acknowledges that all Intellect...
7878    Company software is licensed, not sold. That m...
Name: Clause Text, Length: 7879, dtype: object

In [ ]:
import re
import string
N = int(.15 * len(train_df))
print(N)
preds = []
scores = []
for i, s in enumerate(sentences.values[-N:]):
  
  #if len(s.split()) > 400:
    
    
  #  s = re.sub(r'\d+', '', s)
  #  print(s)
  #  s = ' '.join(re.split(r'\W+', s))
  #  s = " ".join([c for c in s.split() if len(c) > 3])
  #  print(len(s.split()), s)
  #  s = " ".join(s.split()[:400])
  #  print(len(s.split()), s)
  try:
    pred = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)(s)
  except:
    try:
      print('splitting')
      pred1 = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)(s[:len(s)//2])
      pred2 = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)(s[len(s)//2:])
      if pred1[0]['label'] == 'LABEL_1' or pred2[0]['label'] == 'LABEL_1':
        pred = [{'label': 'LABEL_1', 'score': (pred1[0]['score'] + pred2[0]['score']) / 2.}]
      else:
        pred = [{'label': 'LABEL_1', 'score': (pred1[0]['score'] + pred2[0]['score']) / 2.}]
    except:
      pred = [{'label': 'LABEL_1', 'score': .5}]
  if pred[0]['label'] == 'LABEL_0':
    preds.append(0)
    pred[0]['score'] = 1 - pred[0]['score']
  else:
    preds.append(1)
  scores.append(pred[0]['score'])
  print(i, pred)
print(preds)


1181
0 [{'label': 'LABEL_0', 'score': 0.15991109609603882}]
1 [{'label': 'LABEL_0', 'score': 0.004518091678619385}]
2 [{'label': 'LABEL_0', 'score': 0.010115861892700195}]
3 [{'label': 'LABEL_0', 'score': 0.3962051272392273}]
4 [{'label': 'LABEL_0', 'score': 0.009182393550872803}]
5 [{'label': 'LABEL_0', 'score': 0.02454286813735962}]


Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors


6 [{'label': 'LABEL_0', 'score': 0.003848731517791748}]
splitting
7 [{'label': 'LABEL_1', 'score': 0.7450151741504669}]
8 [{'label': 'LABEL_0', 'score': 0.004848361015319824}]
9 [{'label': 'LABEL_0', 'score': 0.013985872268676758}]
10 [{'label': 'LABEL_0', 'score': 0.1416722536087036}]
11 [{'label': 'LABEL_0', 'score': 0.18542402982711792}]
12 [{'label': 'LABEL_0', 'score': 0.005352795124053955}]
13 [{'label': 'LABEL_0', 'score': 0.325817346572876}]
14 [{'label': 'LABEL_1', 'score': 0.8482493162155151}]
15 [{'label': 'LABEL_0', 'score': 0.07074230909347534}]
16 [{'label': 'LABEL_0', 'score': 0.006126224994659424}]
17 [{'label': 'LABEL_0', 'score': 0.1758098602294922}]
18 [{'label': 'LABEL_0', 'score': 0.01659315824508667}]
19 [{'label': 'LABEL_0', 'score': 0.004098355770111084}]
20 [{'label': 'LABEL_0', 'score': 0.009914815425872803}]
21 [{'label': 'LABEL_0', 'score': 0.004665732383728027}]
22 [{'label': 'LABEL_0', 'score': 0.25045716762542725}]
23 [{'label': 'LABEL_0', 'score': 0.0890

Token indices sequence length is longer than the specified maximum sequence length for this model (3645 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1801 > 512). Running this sequence through the model will result in indexing errors


175 [{'label': 'LABEL_0', 'score': 0.020034730434417725}]
splitting
176 [{'label': 'LABEL_1', 'score': 0.5}]
177 [{'label': 'LABEL_0', 'score': 0.2826882600784302}]
178 [{'label': 'LABEL_0', 'score': 0.015490114688873291}]
179 [{'label': 'LABEL_0', 'score': 0.15791690349578857}]
180 [{'label': 'LABEL_0', 'score': 0.06455022096633911}]
181 [{'label': 'LABEL_1', 'score': 0.8097413778305054}]
182 [{'label': 'LABEL_0', 'score': 0.03543555736541748}]
183 [{'label': 'LABEL_0', 'score': 0.37716996669769287}]
184 [{'label': 'LABEL_0', 'score': 0.01129370927810669}]
185 [{'label': 'LABEL_0', 'score': 0.005499839782714844}]
186 [{'label': 'LABEL_0', 'score': 0.05315256118774414}]
187 [{'label': 'LABEL_0', 'score': 0.03244286775588989}]
188 [{'label': 'LABEL_0', 'score': 0.004180967807769775}]
189 [{'label': 'LABEL_0', 'score': 0.012735843658447266}]
190 [{'label': 'LABEL_0', 'score': 0.016965627670288086}]
191 [{'label': 'LABEL_0', 'score': 0.28561049699783325}]
192 [{'label': 'LABEL_0', 'score'

...

1077 [{'label': 'LABEL_0', 'score': 0.049281299114227295}]
splitting
1078 [{'label': 'LABEL_1', 'score': 0.8969943225383759}]
1079 [{'label': 'LABEL_1', 'score': 0.7071120738983154}]
1080 [{'label': 'LABEL_0', 'score': 0.0038417577743530273}]
1081 [{'label': 'LABEL_0', 'score': 0.00389254093170166}]
1082 [{'label': 'LABEL_0', 'score': 0.011592626571655273}]
1083 [{'label': 'LABEL_0', 'score': 0.02076590061187744}]
1084 [{'label': 'LABEL_0', 'score': 0.16608375310897827}]
1085 [{'label': 'LABEL_0', 'score': 0.0632895827293396}]
1086 [{'label': 'LABEL_0', 'score': 0.05055570602416992}]
1087 [{'label': 'LABEL_0', 'score': 0.004154860973358154}]
1088 [{'label': 'LABEL_0', 'score': 0.018082261085510254}]
1089 [{'label': 'LABEL_0', 'score': 0.211134672164917}]
1090 [{'label': 'LABEL_1', 'score': 0.8987536430358887}]
1091 [{'label': 'LABEL_0', 'score': 0.08467048406600952}]
1092 [{'label': 'LABEL_0', 'score': 0.01050180196762085}]
1093 [{'label': 'LABEL_0', 'score': 0.17738652229309082}]
1094

Token indices sequence length is longer than the specified maximum sequence length for this model (885 > 512). Running this sequence through the model will result in indexing errors


1133 [{'label': 'LABEL_0', 'score': 0.3494737148284912}]
splitting
1134 [{'label': 'LABEL_1', 'score': 0.715852677822113}]
1135 [{'label': 'LABEL_1', 'score': 0.6217065453529358}]
1136 [{'label': 'LABEL_0', 'score': 0.030114948749542236}]
1137 [{'label': 'LABEL_0', 'score': 0.07357561588287354}]
1138 [{'label': 'LABEL_1', 'score': 0.9396487474441528}]
1139 [{'label': 'LABEL_0', 'score': 0.00574880838394165}]
1140 [{'label': 'LABEL_0', 'score': 0.01032167673110962}]


Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


1141 [{'label': 'LABEL_0', 'score': 0.010580956935882568}]
splitting
1142 [{'label': 'LABEL_1', 'score': 0.9313382208347321}]
1143 [{'label': 'LABEL_0', 'score': 0.01325768232345581}]
1144 [{'label': 'LABEL_0', 'score': 0.009885907173156738}]
1145 [{'label': 'LABEL_0', 'score': 0.08840519189834595}]
1146 [{'label': 'LABEL_0', 'score': 0.008969366550445557}]
1147 [{'label': 'LABEL_0', 'score': 0.0054590702056884766}]
1148 [{'label': 'LABEL_0', 'score': 0.4110393524169922}]
1149 [{'label': 'LABEL_0', 'score': 0.017261028289794922}]
1150 [{'label': 'LABEL_0', 'score': 0.0049016475677490234}]
1151 [{'label': 'LABEL_0', 'score': 0.042033612728118896}]
1152 [{'label': 'LABEL_0', 'score': 0.020355045795440674}]
1153 [{'label': 'LABEL_1', 'score': 0.6969887018203735}]
1154 [{'label': 'LABEL_1', 'score': 0.8929591178894043}]
1155 [{'label': 'LABEL_1', 'score': 0.5835965871810913}]
1156 [{'label': 'LABEL_0', 'score': 0.1028338074684143}]
1157 [{'label': 'LABEL_0', 'score': 0.0064980387687683105}

In [ ]:
preds

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
print(scores)

[0.0028538107872009277, 0.0002923011779785156, 0.00036603212356567383, 0.011149466037750244, 0.00022530555725097656, 0.0003833770751953125, 0.0002442598342895508, 0.9937028586864471, 0.0004380345344543457, 0.0002970099449157715, 0.0007200241088867188, 0.000603795051574707, 0.00022208690643310547, 0.0002964138984680176, 0.9970483183860779, 0.0003014206886291504, 0.00020653009414672852, 0.013470053672790527, 0.0014649033546447754, 0.00021517276763916016, 0.0003311038017272949, 0.00019806623458862305, 0.956591784954071, 0.0006470680236816406, 0.00022405385971069336, 0.0024265050888061523, 0.0002499818801879883, 0.0002275705337524414, 0.006843447685241699, 0.00022715330123901367, 0.0002471804618835449, 0.0003241300582885742, 0.8905724883079529, 0.972428023815155, 0.0003822445869445801, 0.00024384260177612305, 0.0002662539482116699, 0.00023323297500610352, 0.00022965669631958008, 0.0020992159843444824, 0.00878077745437622, 0.00024765729904174805, 0.08167648315429688, 0.0002027750015258789, 

In [ ]:
targets.values[-N:] 

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
from sklearn.metrics import f1_score

f1_score(targets.values[-len(scores):], preds)

0.5891472868217055

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(targets.values[-N:], preds))
print(classification_report(targets.values[-N:], preds))

[[908  62]
 [ 97 114]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92       970
           1       0.65      0.54      0.59       211

    accuracy                           0.87      1181
   macro avg       0.78      0.74      0.75      1181
weighted avg       0.86      0.87      0.86      1181



In [ ]:
import numpy as np

#brier score 
np.sum((np.array(scores) - targets.values[-len(scores):])**2)/ len(scores)

0.09664510168394609

In [ ]:
from transformers import PretrainedConfig


In [ ]:
config = PretrainedConfig.from_json_file('/content/ai-ml-challenge-2020/models/config.json')

In [ ]:
config

PretrainedConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [ ]:
# f1 and brier score baseline score all zeros 

ImportError: ignored